In [1]:
!pip install easyocr

     |████████████████████████████████| 63.6 MB 10 kB/s 


In [2]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15283, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 15283 (delta 40), reused 50 (delta 26), pack-reused 15186
Receiving objects: 100% (15283/15283), 13.78 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (10351/10351), done.


In [3]:
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/darknet


In [4]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old ==

In [5]:
%cd /content/darknet
import shutil,os
path = '/content/darknet/cfg/'
for i in os.listdir('cfg'):
  i = path + i
  try:
    os.remove(i)
  except IsADirectoryError:
    shutil.rmtree(i)

path = '/content/darknet/data/'
temp = 'labels'
for i in os.listdir('data'):
  if (i!=temp):
    i = path + i
    try:
      os.remove(i)
      print("[INFO] | " + i + " removed")
    except IsADirectoryError:
      shutil.rmtree(i)
      print("[INFO] | " + i + " removed")

/content/darknet
[INFO] | /content/darknet/data/horses.jpg removed
[INFO] | /content/darknet/data/person.jpg removed
[INFO] | /content/darknet/data/imagenet.shortnames.list removed
[INFO] | /content/darknet/data/9k.tree removed
[INFO] | /content/darknet/data/dog.jpg removed
[INFO] | /content/darknet/data/goal.txt removed
[INFO] | /content/darknet/data/coco.names removed
[INFO] | /content/darknet/data/openimages.names removed
[INFO] | /content/darknet/data/coco9k.map removed
[INFO] | /content/darknet/data/scream.jpg removed
[INFO] | /content/darknet/data/imagenet.labels.list removed
[INFO] | /content/darknet/data/giraffe.jpg removed
[INFO] | /content/darknet/data/eagle.jpg removed
[INFO] | /content/darknet/data/voc.names removed


In [8]:
!cp /content/drive/MyDrive/ColabProjects/yolov4-tiny/yolov4-tiny-custom.cfg /content/darknet/cfg

In [9]:
!cp /content/drive/MyDrive/ColabProjects/yolov4-tiny/obj.names /content/darknet/data
!cp /content/drive/MyDrive/ColabProjects/yolov4-tiny/obj.data  /content/darknet/data

In [10]:
!cp /content/drive/MyDrive/ColabProjects/yolov4-tiny/process.py /content/darknet

In [11]:
!python process.py

/content/darknet


In [12]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [13]:
%cd /content/darknet/cfg
!sed -i 's/batch=64/batch=1/' yolov4-tiny-custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-tiny-custom.cfg

/content/darknet/cfg


In [14]:
%cd /content/darknet

/content/darknet


In [15]:
def get_anchor(path):
  import re  
  myfile=open(path,'r')
  lines=myfile.readlines()
  pattern= "license_plate"

  for line in lines:
    if re.search(pattern,line):
      Cord_Raw=line

  Cord=Cord_Raw.split("(")[1].split(")")[0].split("  ")
  x_min=int(Cord[1])
  x_max=x_min + int(Cord[5])
  y_min=int(Cord[3])
  y_max=y_min+ int(Cord[7])
  return x_min,y_min,x_max,y_max

In [16]:
def extract_data(img_path):
  import cv2
  import matplotlib.pyplot as plt
  img = cv2.imread(img_path)
  crop_img = img[y_min:y_max, x_min:x_max]
  cv2.imwrite("/content/Object.jpg",crop_img)
  
  import easyocr
  reader = easyocr.Reader(['en'])
  result = reader.readtext(crop_img)
  print(result)
  
  try:
    text = result[0][1]
    print('Detected text: ' + text)

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = cv2.rectangle(image,(x_min,y_min),(x_max,y_max),(0,255,255),3)

    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (x_min+20, y_max+20)
    fontScale = 0.5
    color = (0, 255, 255)
    thickness = 1
    image = cv2.putText(image,text, org, font, fontScale, color, thickness, cv2.LINE_AA)
    plt.imshow(image)
    plt.show()
  except:
    print('No text detected')

In [ ]:
!./darknet detector demo /content/darknet/data/obj.data /content/darknet/cfg/yolov4-tiny-custom.cfg /content/drive/MyDrive/ColabProjects/yolov4-tiny/training/yolov4-tiny-custom_best.weights -dont_show /content/drive/MyDrive/ColabProjects/yolov4-tiny/test1.mp4 -thresh 0.7 -i 0 -out_filename /content/result.mp4

In [27]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "/content/result.mp4"

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1200 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)